
Task 1

Install the plotly and other data visualization packages using either pip or conda.

In [2]:
#Successfully imports plotly.express
import plotly.express as px
import plotly.io as pio

Task 2

Using the data from Assignment 2, produce at least three types of charts using plotly or other packages of you own preference. For example, you can plot the distribution of state or county areas, the distribution of common county names, and the distribution of the number of counties in states. Directly show those charts in Jupyter Notebook, if applies.

In [3]:
#Import all of the data- most of this code is from assignment 3
import json

with open("gz_2010_us_050_00_5m.json", 'r') as f:
  data = json.load(f)

with open("fipsToState.json", 'r') as f:
    StateFIP_Dict = json.load(f)

featureList = data['features']

#create a dictionary with each state name as the key, and each value being a list of counties stored with unique IDs (GeoID)
StateCounty_Dict = {}
for state in list(StateFIP_Dict.keys()):
    countylist = []
    StateCounty_Dict.update({StateFIP_Dict.get(state):countylist})
    for i in featureList:
        if state == i["properties"]["STATE"]:
            countylist.append(i['properties']['GEO_ID'])
            StateCounty_Dict.update({StateFIP_Dict.get(state):countylist})
        else: 
            pass

In [14]:
#Create a bar chart for number of counties in a state
import pandas as pd

#Create a dictionary that contains the state as the key and the value as the number of counties
StateCountyCount = {}
for state in StateCounty_Dict:
    StateCountyCount.update({state:len(StateCounty_Dict[state])})

#Write the dictionary to a data frame with pandas
df = pd.DataFrame(list(StateCountyCount.items()), columns=['State', 'Counties'])
df = df.sort_values('Counties', ascending=False)

# create bar chart
fig = px.bar(df, x='State', y='Counties',color='State', title='Number of Counties by State', width = 1000 )

# remove legend
fig.update_layout(showlegend=False)

#Write it to a static image (Task 3)

pio.write_image(fig, 'Number_Counties.png')

#Show the bar chart
fig.show()

In [5]:
#Create a scatter plot with each dot representing a state, 
#number of counties is one axis, average county size is the other

#For this graph, we can recycle several functions created in assignments 2 and 3

def DictionaryFactory(key, value):
    TempDictionary = {i['properties'][key]: i['properties'][value] for i in featureList}
    return TempDictionary

CountySize_Dict = DictionaryFactory('GEO_ID','CENSUSAREA')

def AverageCountySize(state):
    counties = StateCounty_Dict.get(state)
    TotalSize = 0 
    for county in counties:
        TotalSize += CountySize_Dict[county]
    MeanSize = TotalSize / len(counties)
    return MeanSize

#Write a new dictionary with the state name as the key, and the value being a list with the number of
#counties and the average county size

CountyDictNumSize = {}
for state in StateCounty_Dict:
    average = AverageCountySize(state)
    values = [StateCountyCount.get(state), average]
    CountyDictNumSize.update({state:values})

#county_df = pd.DataFrame.from_dict(CountyDictNumSize, orient='index', columns=['Counties', 'Avg County Size'])
county_df = pd.DataFrame.from_dict(CountyDictNumSize, orient='index', columns=['Number of Counties', 'Average County Size'])
county_df['State'] = county_df.index

#fig = px.scatter(county_df, x='Avg County Size', y='Counties', title='Number of Counties vs. Average County Size')
fig = px.scatter(county_df, x='Average County Size', y='Number of Counties', color='State', hover_data=['State'], title='Number of Counties vs. Average County Size for U.S. States')

fig.show()

In [31]:
#Create pie chart with each slice representing state size

#create a dictionary with each value being the state name and each key being the size of the state
#Fortunately, we can easily modify and re-use the average county size function. 

def TotalSize(state):
    counties = StateCounty_Dict.get(state)
    Total = 0 
    for county in counties:
        Total += CountySize_Dict[county]
    return Total

StateSizes = {}
for state in StateCounty_Dict:
    size = TotalSize(state)
    StateSizes.update({state:size})

df = pd.DataFrame(list(StateSizes.items()), columns=['State', 'State Size'])
fig = px.pie(df, values='State Size', names='State', title='Size of U.S. States')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
#Write to an html file (Task 3)
fig.write_html('StateSizePieChart.html')
fig.show()


Task 3

Export at least one chart to a static image and one chart to an HTML page, using Python code.


Note: Since the pie chart is difficult to read in the jupyter notebook, it is also exported to an html page. The "Number of Counties by State" chart is exported as a static image